In [1]:
from story_sage import *
import os
import glob
from story_sage.utils.raptor import _RaptorResults
from pprint import pprint
import logging
import markdown
from IPython.display import HTML

logging.basicConfig(level=logging.WARNING)

logger = logging.getLogger('story_sage_chain')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

os.environ['TOKENIZERS_PARALLELISM'] = "false"

config_path = './config.yml'
ssconfig = StorySageConfig.from_file(config_path)
selected_series = next(series for series in ssconfig.series if series.series_metadata_name == 'lord_of_the_rings')


retriever = StorySageRetriever(chroma_path=ssconfig.chroma_path, chroma_collection_name=ssconfig.raptor_collection, n_chunks=15)

In [2]:
conversation = StorySageConversation()

state = StorySageState(
    question=None,
    book_number=4,
    chapter_number=1,
    series_id=selected_series.series_id,
    conversation=conversation,
)
chain = StorySageChain(config=ssconfig, state=state)
chain.logger.setLevel(logging.INFO)

def ask_question(question: str):
    chain.state.question = question
    result = chain.invoke()
    conversation.add_turn(question=question, response=result.answer, context=result.context)
    chain.state.conversation = conversation
    return HTML(f'<div style="max-width:1000px;">{markdown.markdown(result.answer)}</div>')

In [3]:
ask_question("""
             Why does Aragorn choose to follow the Orcs and not go after Frodo? Why is that choice so painful? In what ways might our more difficult and painful choices be the most
important?
             """)

INFO:story_sage.services.story_sage_chain:Invoking StorySageChain with question: 
             Why does Aragorn choose to follow the Orcs and not go after Frodo? Why is that choice so painful? In what ways might our more difficult and painful choices be the most
important?
             
INFO:story_sage.services.story_sage_chain:Context filters: {'entities': [], 'series_id': 8, 'book_number': 4, 'chapter_number': 1}


In [4]:
pprint(chain.state)
print(chain.state.get_cost())

StorySageState(question='\n'
                        '             Why does Aragorn choose to follow the '
                        'Orcs and not go after Frodo? Why is that choice so '
                        'painful? In what ways might our more difficult and '
                        'painful choices be the most\n'
                        'important?\n'
                        '             ',
               book_number=4,
               chapter_number=1,
               series_id=8,
               summary_chunks=None,
               context_filters={'book_number': 4,
                                'chapter_number': 1,
                                'entities': [],
                                'series_id': 8},
               initial_context=[],
               target_ids=[],
               context=[StorySageContext(chunk_id='series_8|book_1|chapter_14|level_2|chunk_0',
                                         book_number=1,
                                         chapter_number=1

In [5]:
ask_question("yes, is he ever tempted to take the ring for himself?")

INFO:story_sage.services.story_sage_chain:Invoking StorySageChain with question: yes, is he ever tempted to take the ring for himself?
INFO:story_sage.services.story_sage_chain:Context filters: {'entities': [], 'series_id': 8, 'book_number': 4, 'chapter_number': 1}
INFO:story_sage.services.story_sage_chain:Question is a followup
INFO:story_sage.services.story_sage_chain:Refined follow-up question: Is Aragorn ever tempted to take the One Ring for himself during the quest?
INFO:story_sage.services.story_sage_chain:Getting context from followup query


In [6]:
pprint(chain.state)
print(chain.state.get_cost())

StorySageState(question='Is Aragorn ever tempted to take the One Ring for '
                        'himself during the quest?',
               book_number=4,
               chapter_number=1,
               series_id=8,
               summary_chunks=None,
               context_filters={'book_number': 4,
                                'chapter_number': 1,
                                'entities': [],
                                'series_id': 8},
               initial_context=[],
               target_ids=[],
               context=[StorySageContext(chunk_id='series_8|book_1|chapter_14|level_1|chunk_3',
                                         book_number=1,
                                         chapter_number=14,
                                         chunk='I have seen three ages in the '
                                               'West of the world, and many '
                                               'defeats, and many fruitless '
                               

In [ ]:
where = {
    '$and': [
        {'$or': [
            {'book_number': {'$lt': 4}}, 
            {'$and': [
                {'book_number': 4}, 
                {'chapter_number': {'$lt': 1}}
            ]}
        ]}, 
        {'series_id': 8}, 
        {'parents': ''}
    ]}

res = retriever.vector_store.query(query_texts="Boromir character arc throughout the books, including his nobility, struggle with the One Ring, sacrifice, and redemption",
                             where=where)

res['ids'][0]
                            

['series_8|book_2|chapter_1|level_2|chunk_0',
 'series_8|book_2|chapter_16|level_3|chunk_0',
 'series_8|book_2|chapter_2|level_3|chunk_0',
 'series_8|book_1|chapter_22|level_2|chunk_0',
 'series_8|book_3|chapter_9|level_2|chunk_0',
 'series_8|book_1|chapter_14|level_3|chunk_0',
 'series_8|book_3|chapter_1|level_3|chunk_0',
 'series_8|book_3|chapter_2|level_3|chunk_0',
 'series_8|book_3|chapter_5|level_2|chunk_0',
 'series_8|book_1|chapter_15|level_3|chunk_0']

In [6]:
pprint(chain.state)
print(chain.state.get_cost())

StorySageState(question='no, just tell me about what they believe in and how '
                        'it influences their behavior',
               book_number=11,
               chapter_number=1,
               series_id=3,
               summary_chunks=None,
               context_filters={'book_number': 11,
                                'chapter_number': 1,
                                'entities': [],
                                'series_id': 3},
               initial_context=[],
               target_ids=[],
               context=[StorySageContext(chunk_id='series_3|book_10|chapter_28|level_2|chunk_0',
                                         book_number=10,
                                         chapter_number=28,
                                         chunk='In the context, a character '
                                               'reflects on the significance '
                                               'of status and relationships, '
                     

In [7]:
ask_question("do they eat meat?")

In [7]:
pprint(chain.state.context)

[StorySageContext(chunk_id='series_3|book_1|chapter_43|level_1|chunk_5',
                  book_number=1,
                  chapter_number=43,
                  chunk='In the beginning. This contest between us has taken '
                        'place countless times before. Each time your face is '
                        'different, and your name, but each time it is you.” I '
                        'deny you.” It was a desperate whisper. Each time you '
                        'throw your puny strength against me, and each time, '
                        'in the end, you know which of us is the master. Age '
                        'after Age, you kneel to me, or die wishing you still '
                        'had strength to kneel. Poor fool, you can never win '
                        'against me.” Liar!” he shouted. “Father of Lies. '
                        'Father of Fools if you can’t do better than that. Men '
                        'found you in the last Age, in the Age 